# Importing Libraries
in this part we will install all the necessary libraries on command prompt and then import the necessary functions from those libraries. 

In [ ]:
# importing all the necessary libraries
import pandas as pd

from numpy import mean
import numpy as np
import time

# step 1: preprocessing
from sklearn.impute import SimpleImputer # import some strategic imputer to fill in any missing values using mean
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, Normalizer # scale all the values to one range to avoid any biasness (this bias is seen in mostly naive bayes and knn etc)

from sklearn.impute import KNNImputer # import some strategic imputer to fill missing values using KNN (finds the nearest neighbour and fills it with that value)

from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, f_classif, VarianceThreshold
from sklearn.feature_selection import f_regression
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso

# step 2: data division
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, GridSearchCV, ParameterGrid # to divide the code into train/test using a specific percentage or with/without replacement

# step 3: model
from xgboost import XGBRegressor

# step 4: displaying accuracy
from sklearn.metrics import roc_auc_score, accuracy_score # to display the accuracy of our tree
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer

# step 5: warning filter
import warnings
warnings.filterwarnings('ignore')

In [108]:
from datetime import datetime

# Function to get current date and time as a string
def get_current_datetime():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Data Loading
data shall be loaded into variables as data sets using pandas and csv readers. they will be checked to see if they are loaded properly and will be loaded as 2 sets: train and test as per given in the kaggle data

In [109]:
# lets load the training data set
train_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger2\iml-fall-2024-challenge-2\train\train.csv")

# lets also check it by getting the first few rows of the data, there should be x1 - x78 and one target variable Y
train_data.head() 

,full_sq,life_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,43.0,27.0,4.0,Investment,Bibirevo,6407578.100,155572.0,0.189727,0.000070,9576.0,...,9.0,4.0,0.0,13.0,22.0,1.0,0.0,52.0,4.0,5850000.0
1,34.0,19.0,3.0,Investment,Nagatinskij Zaton,9589336.912,115352.0,0.372602,0.049637,6880.0,...,15.0,3.0,0.0,15.0,29.0,1.0,10.0,66.0,14.0,6000000.0
2,43.0,29.0,2.0,Investment,Tekstil'shhiki,4808269.831,101708.0,0.112560,0.118537,5879.0,...,10.0,3.0,0.0,11.0,27.0,0.0,4.0,67.0,10.0,5700000.0
3,77.0,77.0,4.0,Investment,Basmannoe,8398460.622,108171.0,0.015234,0.037316,5706.0,...,319.0,108.0,17.0,135.0,236.0,2.0,91.0,195.0,14.0,16331452.0
4,67.0,46.0,14.0,Investment,Nizhegorodskoe,7506452.020,43795.0,0.007670,0.486246,2418.0,...,62.0,14.0,1.0,53.0,78.0,1.0,20.0,113.0,17.0,9100000.0


In [110]:
# lets load the test data
test_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger2\iml-fall-2024-challenge-2\test\test.csv")

# check if the data has been loaded by getting the first 5 rows - there should be x1 - x78 and no target variable Y as this is test data
test_data.head() 

,row ID,full_sq,life_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,Row3,89.0,50.0,9.0,Investment,Mitino,1.258354e+07,178473.0,0.194703,0.069753,...,15.0,11.0,2.0,1.0,4.0,4.0,0.0,0.0,26.0,3.0
1,Row6,25.0,14.0,10.0,Investment,Sokol'niki,1.032047e+07,57405.0,0.523439,0.042307,...,144.0,81.0,16.0,3.0,38.0,80.0,1.0,27.0,127.0,8.0
2,Row11,38.0,19.0,11.0,Investment,Zapadnoe Degunino,7.632940e+06,78810.0,0.051844,0.437885,...,39.0,8.0,3.0,0.0,10.0,9.0,0.0,0.0,35.0,4.0
3,Row12,43.0,28.0,4.0,Investment,Kuncevo,5.235177e+07,142462.0,0.070662,0.035145,...,21.0,13.0,9.0,1.0,7.0,15.0,0.0,2.0,47.0,0.0
4,Row14,31.0,21.0,3.0,Investment,Lefortovo,8.993640e+06,89971.0,0.066941,0.306977,...,205.0,88.0,19.0,2.0,63.0,100.0,0.0,28.0,132.0,14.0


# Data Preprocessing
before we start processing this data and using algorithms, we will fix this data first, this is called data preprocessing

## split data into categorical and numerical
categorical will have one-hot and simple imputer of most frequent while numerical will have simple mean imputer and minmax scaler

In [111]:
categorical_cols = train_data.select_dtypes(include=["object"]).columns
numerical_cols = train_data.select_dtypes(exclude=["object"]).drop(columns=['price_doc']).columns

In [112]:
num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

In [113]:
# Column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, numerical_cols),
        ("cat", cat_transformer, categorical_cols)
    ]
)

## correlation matrix
i tried getting the correlation matrix but apparently a 2000 columns matrix is very computationally expensive as it performs pairs for all. so dont run it. it takes too long and then fails. i ran for 5 minutes. 

In [114]:
# # DONT RUN
# corr_matrix = train_data.corr()
# print(corr_matrix)

# PCA
principal component analysis is applied

In [115]:
# # -------------------------- case  --------------------------
# pca = PCA(n_components=33)                                 
# X = pca.fit_transform(X)
# test_data_processed = pca.transform(test_data_processed)

## Data Splitting - festures and targets
the data in train_data set is of x1 - x78 columns (79 variables) and one target variable (Y). we must split that data so that we can perform data preprocessing on the features variables (will be referred to as X).

In [116]:
# so in X, it is ALL the columns EXCEPT the last column known as 'Y' (we can confirm this using the train_data.head() we did earlier) so we must get all columns and DROP only the 'y' column
X = train_data.drop(columns=['price_doc'])
X # lets display X and see what it is now

,full_sq,life_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,43.0,27.0,4.0,Investment,Bibirevo,6.407578e+06,155572.0,0.189727,0.000070,9576.0,...,40.0,9.0,4.0,0.0,13.0,22.0,1.0,0.0,52.0,4.0
1,34.0,19.0,3.0,Investment,Nagatinskij Zaton,9.589337e+06,115352.0,0.372602,0.049637,6880.0,...,36.0,15.0,3.0,0.0,15.0,29.0,1.0,10.0,66.0,14.0
2,43.0,29.0,2.0,Investment,Tekstil'shhiki,4.808270e+06,101708.0,0.112560,0.118537,5879.0,...,25.0,10.0,3.0,0.0,11.0,27.0,0.0,4.0,67.0,10.0
3,77.0,77.0,4.0,Investment,Basmannoe,8.398461e+06,108171.0,0.015234,0.037316,5706.0,...,552.0,319.0,108.0,17.0,135.0,236.0,2.0,91.0,195.0,14.0
4,67.0,46.0,14.0,Investment,Nizhegorodskoe,7.506452e+06,43795.0,0.007670,0.486246,2418.0,...,155.0,62.0,14.0,1.0,53.0,78.0,1.0,20.0,113.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181502,48.0,33.0,3.0,Investment,Poselenie Mihajlovo-Jarcevskoe,6.455617e+07,4949.0,0.586175,0.005819,346.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
181503,48.0,33.0,3.0,Investment,Poselenie Mihajlovo-Jarcevskoe,6.455617e+07,4949.0,0.586175,0.005819,346.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
181504,48.0,33.0,3.0,Investment,Poselenie Mihajlovo-Jarcevskoe,6.455617e+07,4949.0,0.586175,0.005819,346.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
181505,48.0,33.0,3.0,Investment,Poselenie Mihajlovo-Jarcevskoe,6.455617e+07,4949.0,0.586175,0.005819,346.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [117]:
# so as per our X output, we can see that number of columns in train_data is 79 and number of columns in X is 78 meaning we have successfully performed our removal of target variable
# now to get the target variable alone, we can just get it alone,
Y = train_data['price_doc']
Y # lets see what it is
# as per our Y output, we can see it is of one column and 246k rows which means we have successfully extracted the target variable column

0          5850000.0
1          6000000.0
2          5700000.0
3         16331452.0
4          9100000.0
             ...    
181502     3480000.0
181503     3480000.0
181504     3480000.0
181505     3480000.0
181506     3480000.0
Name: price_doc, Length: 181507, dtype: float64

# Filters
there are two types of filters to filter out columns/features:
- variance filter (a column which has same values throughout the column like all are sunny)
- correlation filter (two columns which are same like weight in kg and weight in pounds)

In [118]:
# print("X : ", X.shape)
# print("test data : ", test_data_processed.shape)

In [119]:
# variance filter
# ----------------------------- case  -----------------------------
# variance_filter = VarianceThreshold(threshold=0.001)  # Adjust the threshold if needed
# X = variance_filter.fit_transform(X)
# test_data_processed = variance_filter.fit_transform(test_data_processed)
X.shape

(181507, 271)

In [120]:
# test_data_processed.shape

In [121]:
# # correlation filter
# # ----------------------------- case  -----------------------------
# corr_matrix = pd.DataFrame(X).corr().abs()
# upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]
# X = pd.DataFrame(X).drop(columns=to_drop)
# test_data_processed = pd.DataFrame(test_data_processed).drop(columns=to_drop)
X.shape

(181507, 271)

In [122]:
# test_data_processed.shape

## Data Splitting - train and validate
now our test_data set is of rows with NO target variable whereas the train_data set is WITH target variable.
our rules in machine learning is that we must train half or 70% of the data and then we must check its accuracy using the remaining half or 30% of the data - we can only check accuracy IF we have the answers i.e. the target variable. 
So, what we need to do is, is split the train_data set into 2, by a 70% and 30% ratio. we train the model using the 70% and then test the model using the 30% and then use that model to predict the test_data set.

In [123]:
# holdout method
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=42)

# functions
here we have defined functions like forward-backward selection, kbest selection & algorithm feature importance

In [ ]:
# forward backward selection
def fbselection(direction, sample_model, features, X, trainX, trainY, testX, test_data_processed):
    print("starting")
    selection = SequentialFeatureSelector(sample_model, direction=direction, n_features_to_select=features, scoring='roc_auc')
    return modelSelector(sample_model, selection, X, trainX, trainY, testX, test_data_processed)

def modelSelector(sample_model, selection, X, trainX, trainY, testX, test_data_processed):
    print("start extracting")
    trainX = selection.fit_transform(trainX, trainY)
    print("extracted, transforming")
    testX = selection.transform(testX)                                  # Ensure the test set is transformed similarly
    test_data_processed = selection.transform(test_data_processed)      # test data is also transformed
    X = selection.transform(X)                                          # full data transforming
    print("transformed")
    return sample_model, X, trainX, trainY, testX, test_data_processed

# kbest selection
def kbest(sample_model, features, X, trainX, trainY, testX, test_data_processed):
    print("starting")
    selection = SelectKBest(score_func=f_regression, k=features)
    return modelSelector(sample_model, selection, X, trainX, trainY, testX, test_data_processed)

In [ ]:
# feature importance function
def featureImportance(sample_model, features, X, trainX, trainY, testX, test_data_processed):
    print("fitting")
    
    # fit the model
    sample_model.fit(trainX, trainY)

    print("extracting features")

    # extract all the feature names from data
    importances = sample_model.feature_importances_
    feature_names = train_data.drop(columns=['Y']).columns
    print(feature_names)

    # sort with respect to importance
    feature_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)

    # extract the top ones
    top_features = feature_importance_df['Feature'].head(features).values
    print(top_features)

    # change all data according to the top ones we have selected
    trainX = pd.DataFrame(trainX, columns=feature_names)[top_features]
    testX = pd.DataFrame(testX, columns=feature_names)[top_features]
    X = pd.DataFrame(X, columns=feature_names)[top_features]
    test_data_processed = pd.DataFrame(test_data_processed, columns=feature_names)[top_features]

    print("features extracted")
    
    # retrain the model
    sample_model.fit(trainX, trainY)

    print("features trained")
    
    return sample_model, X, trainX, trainY, testX, test_data_processed

## model intialization
here model is intialized

In [126]:
trainX = preprocessor.fit_transform(trainX)
print("trainX completed")
testX = preprocessor.transform(testX)
print("testX completed")
test_data = preprocessor.transform(test_data)
print("test data completed")
X = preprocessor.transform(X)
print(X.shape)

trainX completed
testX completed
test data completed
(181507, 2214)


In [127]:
# declare the model here
model = XGBRegressor(max_depth=10, learning_rate=0.01, n_estimators=1000, subsample=0.8, colsample_bytree=0.8, reg_lambda=1, reg_alpha=0, random_state=42)

In [128]:
print("X shape -> ", X.shape)
print("trainX shape -> ", trainX.shape)
print("testX shape -> ", testX.shape)
print("test_data_processed shape -> ", test_data.shape)

X shape ->  (181507, 2214)
trainX shape ->  (145205, 2214)
testX shape ->  (36302, 2214)
test_data_processed shape ->  (77789, 2214)


# feature selection
here we will apply feature selection and feature importance

In [129]:
# Feature selection
# model, X, trainX, trainY, testX, test_data = kbest(
#     model, 40, X, trainX, trainY, testX, test_data
# )

In [130]:
print("X shape -> ", X.shape)
print("trainX shape -> ", trainX.shape)
print("testX shape -> ", testX.shape)
print("test_data_processed shape -> ", test_data.shape)

X shape ->  (181507, 2214)
trainX shape ->  (145205, 2214)
testX shape ->  (36302, 2214)
test_data_processed shape ->  (77789, 2214)


# grid search

In [131]:
def gridsearch(param_grid, model, scorer, trainX, trainY):
    print("starting grid search")

    # intialize grid search
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring=scorer, verbose=3)
    print("grid search is intialized")

    # fit the model
    grid_search.fit(trainX, trainY)
    print("grid search fitting completed")

    # display the best model grid search found
    best_model = grid_search.best_estimator_
    print(best_model)

    # display the best parameters of the best model
    best_parameters = grid_search.best_params_
    print(best_parameters)

    # display the best score of the best model
    print("Best cross-validated score:", grid_search.best_score_)

    # assign the best model our model
    model = best_model
    print("model assigned, grid search completed")

    return model

In [132]:
# # Define a scoring metric (e.g., negative mean squared error)
# scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [133]:
# # define hyper parameters of grid
# param_grid = {
#     'estimators': [10, 100, 200, 500, 1000, 2000, 3000], 
#     'learning_rate': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5],
#     'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# }

In [134]:
# sample from train_data
# sample_train = train_data.sample(frac=0.5)
# sample_X = sample_train.drop('price_doc', axis=1) 
# sample_Y = sample_train['price_doc']

In [135]:
# model = gridsearch(param_grid, model, scorer, trainX, trainY)

## model running
here we run the model

In [136]:
# compute this predictions metrics
def metrics(y_pred, testY):
    print("starting to compute metrics")
    
    # display the mean squared error of this prediction
    mse = mean_squared_error(testY, y_pred)
    print("Mean squared error: %.2f" % mse, "   ")

    # display the root mean squared error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    print("Root Mean squared error: %.2f" % rmse, "   ")

    # display the mean absolute error of this prediction
    mae = mean_absolute_error(testY, y_pred)
    print("Mean absolute error: %.2f" % mae, "   ")

    # display the coeffeicient of determination of this preduction
    r2_Score = r2_score(testY, y_pred)
    print("Coefficient of determination: %.2f" % r2_Score, "    ")
    
    return rmse

In [137]:
def run_model(model, trainX, trainY, testX, testY):
    print("training model")
    model.fit(trainX, trainY)
    print("computing score")
    print("model score: ", model.score(trainX, trainY))
    y_pred = model.predict(testX)
    rmse = metrics(y_pred, testY)
    return model, rmse

In [138]:
best_rmse = 1000000000000000000000000000
best_model = None
best_X = None
best_test_data = None
best_n = 0

In [139]:
for n in range(50, 100):
    print("  iteration ", n, " ", get_current_datetime())
    model, i_X, i_trainX, i_trainY, i_testX, i_test_data = kbest(model, n, X, trainX, trainY, testX, test_data)
    model, rmse = run_model(model, i_trainX, i_trainY, i_testX, testY)
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_X = i_X
        best_test_data = i_test_data
        best_n = n
        print("BESTTTTTT")

  iteration  50   2024-11-27 10:02:17
starting
start extracting


extracted, transforming
transformed
training model
computing score
model score:  0.9231895176564167
starting to compute metrics
Mean squared error: 160812738593420.28    
Root Mean squared error: 12681196.26    
Mean absolute error: 5445754.96    
Coefficient of determination: 0.67     
BESTTTTTT
  iteration  51   2024-11-27 10:03:44
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.9236161807528541
starting to compute metrics
Mean squared error: 160726106860931.44    
Root Mean squared error: 12677780.04    
Mean absolute error: 5437668.61    
Coefficient of determination: 0.67     
BESTTTTTT
  iteration  52   2024-11-27 10:05:11
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.9245219931476936
starting to compute metrics
Mean squared error: 160488472346401.91    
Root Mean squared error: 12668404.49    
Mean absolute error: 5434565.59    
Coefficient of determination

In [140]:
print("best was ", best_n, " ", best_rmse, "     ")

best was  88   12628576.410981964      


In [141]:
for n in range(100, 150):
    print("  iteration ", n, " ", get_current_datetime())
    model, i_X, i_trainX, i_trainY, i_testX, i_test_data = kbest(model, n, X, trainX, trainY, testX, test_data)
    model, rmse = run_model(model, i_trainX, i_trainY, i_testX, testY)
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_X = i_X
        best_test_data = i_test_data
        best_n = n

print("best was ", best_n, " ", best_rmse, "     ")

  iteration  100   2024-11-27 11:28:59
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.9383456380667085
starting to compute metrics
Mean squared error: 160139122739112.44    
Root Mean squared error: 12654608.75    
Mean absolute error: 5381091.31    
Coefficient of determination: 0.67     
  iteration  101   2024-11-27 11:31:26
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.937715134726815
starting to compute metrics
Mean squared error: 160718892325865.25    
Root Mean squared error: 12677495.51    
Mean absolute error: 5386817.71    
Coefficient of determination: 0.67     
  iteration  102   2024-11-27 11:33:31
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.938306492434976
starting to compute metrics
Mean squared error: 160629784727666.66    
Root Mean squared error: 12673980.62    
Mean absolute error

In [142]:
for n in range(150, 200):
    print("  iteration ", n, " ", get_current_datetime())
    model, i_X, i_trainX, i_trainY, i_testX, i_test_data = kbest(model, n, X, trainX, trainY, testX, test_data)
    model, rmse = run_model(model, i_trainX, i_trainY, i_testX, testY)
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_X = i_X
        best_test_data = i_test_data
        best_n = n

print("best was ", best_n, " ", best_rmse, "     ")

  iteration  150   2024-11-27 14:23:43
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.9431914361632976
starting to compute metrics
Mean squared error: 160744662742773.06    
Root Mean squared error: 12678511.85    
Mean absolute error: 5370699.71    
Coefficient of determination: 0.67     
  iteration  151   2024-11-27 14:30:49
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.9439298060195264
starting to compute metrics
Mean squared error: 160802492791782.81    
Root Mean squared error: 12680792.28    
Mean absolute error: 5377838.26    
Coefficient of determination: 0.67     
  iteration  152   2024-11-27 14:36:34
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.9428985571416577
starting to compute metrics
Mean squared error: 160721599714592.53    
Root Mean squared error: 12677602.29    
Mean absolute err

In [143]:
for n in range(200, 250):
    print("  iteration ", n, " ", get_current_datetime())
    model, i_X, i_trainX, i_trainY, i_testX, i_test_data = kbest(model, n, X, trainX, trainY, testX, test_data)
    model, rmse = run_model(model, i_trainX, i_trainY, i_testX, testY)
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_X = i_X
        best_test_data = i_test_data
        best_n = n

print("best was ", best_n, " ", best_rmse, "     ")

  iteration  200   2024-11-27 18:39:30
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.9486634454578426
starting to compute metrics
Mean squared error: 160895358347019.84    
Root Mean squared error: 12684453.41    
Mean absolute error: 5367180.23    
Coefficient of determination: 0.67     
  iteration  201   2024-11-27 18:47:56
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.9478416753481571
starting to compute metrics
Mean squared error: 160225972368186.25    
Root Mean squared error: 12658039.83    
Mean absolute error: 5356904.34    
Coefficient of determination: 0.67     
  iteration  202   2024-11-27 18:53:10
starting
start extracting
extracted, transforming
transformed
training model
computing score
model score:  0.9484265951817002
starting to compute metrics
Mean squared error: 159877013684127.66    
Root Mean squared error: 12644248.25    
Mean absolute err

In [144]:
X = best_X
test_data = best_test_data

## predict for test dataset
fit the model and predict for test dataset

In [145]:
model.fit(X, Y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [146]:
# display information regarding the regression
print("model score: ", model.score(X, Y), "    ")
# print("model coefficient: ", model.coef_)
# print("model intercept: ", model.intercept_)

model score:  0.9200141191638683     


In [147]:
test_prediction = model.predict(test_data)

# test_prediction=test_prediction[:, 1]

print(test_prediction)

[12782657.   5567961.   5022173.  ...  3583496.8  3583496.8  3583496.8]


## write into csv
now we write the predictions into the csv file

In [148]:
sample_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger2\iml-fall-2024-challenge-2\sample_submission.csv")

sample_data['price_doc'] = test_prediction
sample_data

sample_data.to_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger2\iml-fall-2024-challenge-2\xgb1.csv", index=False)
sample_data

,row ID,price_doc
0,Row3,12782657.00
1,Row6,5567961.00
2,Row11,5022173.00
3,Row12,5823963.00
4,Row14,5315918.50
...,...,...
77784,Row18591dupl_228801,63785708.00
77785,Row18591dupl_228803,61245944.00
77786,Row18591dupl_228814,3583496.75
77787,Row18591dupl_228817,3583496.75


In [149]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)